In [53]:
%%capture
!pip install -r requirements-components-test.txt

In [54]:
import torch
import numpy as np

In [55]:
np.__version__

'1.24.3'

In [56]:
from torch import nn

In [57]:
torch.__version__

'2.3.0+cu121'

In [58]:
import pandas as pd

In [59]:
pd.__version__

'2.0.3'

In [60]:
import os
import requests
import logging
# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configuration variables
config = {
    "REPO_URL": os.environ.get('REPO_URL', 'https://github.com/danilonicioka/mlops-workflow.git'),
    "CLONED_DIR": os.environ.get('CLONED_DIR', 'mlops-workflow'),
    "FILE_URL": os.environ.get('FILE_URL', 'https://raw.githubusercontent.com/razaulmustafa852/youtubegoes5g/main/Models/Stall-Windows%20-%20Stall-3s.csv'),
    "DVC_FILE_DIR": os.environ.get('DVC_FILE_DIR', 'data/external'),
    "DVC_FILE_NAME": os.environ.get('DVC_FILE_NAME', 'init_dataset.csv'),
    "BRANCH_NAME": os.environ.get('BRANCH_NAME', 'tests'),
    "BUCKET_NAME": os.environ.get('BUCKET_NAME', 'dvc-data'),
    "MINIO_URL": os.environ.get('MINIO_URL', 'localhost:9000'),
    "ACCESS_KEY": os.environ.get('ACCESS_KEY'),
    "SECRET_KEY": os.environ.get('SECRET_KEY'),
    "REMOTE_NAME": os.environ.get('REMOTE_NAME', 'minio_remote'),
    "GITHUB_USERNAME": os.environ.get('GITHUB_USERNAME'),
    "GITHUB_TOKEN": os.environ.get('GITHUB_TOKEN')
}

file_url = config["FILE_URL"]
local_file_path = config["DVC_FILE_NAME"]

try:
    # Request the file content
    response = requests.get(file_url)
    response.raise_for_status()

    # Save the file content locally
    with open(local_file_path, 'wb') as local_file:
        local_file.write(response.content)
    logger.info(f"Successfully downloaded file from {file_url} to {local_file_path}")
except requests.RequestException as e:
    # Log and raise any download errors
    logger.error(f"Failed to download file: {e}")
    raise

INFO:__main__:Successfully downloaded file from https://raw.githubusercontent.com/razaulmustafa852/youtubegoes5g/main/Models/Stall-Windows%20-%20Stall-3s.csv to init_dataset.csv


In [61]:
#colab_path = os.path.join('/content', local_file_path)
df = pd.read_csv(local_file_path)

In [62]:
df.columns

Index(['ID', 'Stall', 'Quality', 'Time', 'CQI1', 'CQI2', 'CQI3', 'cSTD CQI',
       'cMajority', 'c25 P', 'c50 P', 'c75 P', 'RSRP1', 'RSRP2', 'RSRP3',
       'pMajority', 'p25 P', 'p50 P', 'p75 P', 'RSRQ1', 'RSRQ2', 'RSRQ3',
       'qMajority', 'q25 P', 'q50 P', 'q75 P', 'SNR1', 'SNR2', 'SNR3',
       'sMajority', 's25 P', 's50 P', 's75 P'],
      dtype='object')

In [63]:
df = df.replace([' ', '-',np.nan], 0) # There are null values
#df = df.replace([' ', '-',np.nan], np.nan)

In [64]:
# Selective columns for mean calculation
columns_to_convert = ['CQI1', 'CQI2', 'CQI3', 'cSTD CQI',
       'cMajority', 'c25 P', 'c50 P', 'c75 P', 'RSRP1', 'RSRP2', 'RSRP3',
       'pMajority', 'p25 P', 'p50 P', 'p75 P', 'RSRQ1', 'RSRQ2', 'RSRQ3',
       'qMajority', 'q25 P', 'q50 P', 'q75 P', 'SNR1', 'SNR2', 'SNR3',
       'sMajority', 's25 P', 's50 P', 's75 P']
df[columns_to_convert] = df[columns_to_convert].astype(float)

# Replace np.nan with mean values for selective columns
df[columns_to_convert] = df[columns_to_convert].fillna(df[columns_to_convert].mean())

# Display the modified DataFrame
#print(df)

In [65]:
# Check which columns contain np.nan values
columns_with_nan = df.isna().any()
# Display the columns with np.nan values
print(columns_with_nan)

ID           False
Stall        False
Quality      False
Time         False
CQI1         False
CQI2         False
CQI3         False
cSTD CQI     False
cMajority    False
c25 P        False
c50 P        False
c75 P        False
RSRP1        False
RSRP2        False
RSRP3        False
pMajority    False
p25 P        False
p50 P        False
p75 P        False
RSRQ1        False
RSRQ2        False
RSRQ3        False
qMajority    False
q25 P        False
q50 P        False
q75 P        False
SNR1         False
SNR2         False
SNR3         False
sMajority    False
s25 P        False
s50 P        False
s75 P        False
dtype: bool


In [66]:
df['Stall'].replace('Yes', 1, inplace=True)
df['Stall'].replace('No', 0, inplace=True)

In [67]:
print(df)

          ID  Stall   Quality      Time  CQI1  CQI2  CQI3  cSTD CQI  \
0      4P7s2      0    hd2160  16:14:29  13.0  13.0  13.0  0.000000   
1      4P7s2      0    hd2160  16:14:30  13.0  13.0  13.0  0.000000   
2      4P7s2      0    hd2160  16:14:31  13.0  13.0  13.0  0.000000   
3      4P7s2      0    hd2160  16:14:32  13.0  13.0  12.0  0.471405   
4      4P7s2      0    hd2160  16:14:33  12.0  14.0  12.0  0.942809   
...      ...    ...       ...       ...   ...   ...   ...       ...   
2688  5Po26s      1  unknown,  17:43:18  14.0  14.0  14.0  0.000000   
2689  4Po26s      1  unknown,  17:43:23   0.0   0.0   0.0  0.000000   
2690  4Po26s      1   hd1440,  17:43:33   0.0   0.0   0.0  0.000000   
2691   4I27s      1  unknown,  10:52:04   9.0   9.0   9.0  0.000000   
2692   4I27s      1   hd1440,  10:52:16   9.0   9.0   9.0  0.000000   

      cMajority  c25 P  ...  q25 P  q50 P  q75 P  SNR1  SNR2  SNR3  sMajority  \
0          13.0   13.0  ...   -9.5   -7.0   -7.0  12.0  12.0   7.0

In [68]:
df.columns

Index(['ID', 'Stall', 'Quality', 'Time', 'CQI1', 'CQI2', 'CQI3', 'cSTD CQI',
       'cMajority', 'c25 P', 'c50 P', 'c75 P', 'RSRP1', 'RSRP2', 'RSRP3',
       'pMajority', 'p25 P', 'p50 P', 'p75 P', 'RSRQ1', 'RSRQ2', 'RSRQ3',
       'qMajority', 'q25 P', 'q50 P', 'q75 P', 'SNR1', 'SNR2', 'SNR3',
       'sMajority', 's25 P', 's50 P', 's75 P'],
      dtype='object')

In [69]:
X = df[['CQI1', 'CQI2', 'CQI3', 'cSTD CQI',
       'cMajority', 'c25 P', 'c50 P', 'c75 P', 'RSRP1', 'RSRP2', 'RSRP3',
       'pMajority', 'p25 P', 'p50 P', 'p75 P', 'RSRQ1', 'RSRQ2', 'RSRQ3',
       'qMajority', 'q25 P', 'q50 P', 'q75 P', 'SNR1', 'SNR2', 'SNR3',
       'sMajority', 's25 P', 's50 P', 's75 P']].values

In [70]:
y = df['Stall'].values

In [71]:
X.shape, y.shape

((2693, 29), (2693,))

In [72]:
import numpy as np

In [73]:
import sklearn
from sklearn.model_selection import train_test_split

In [74]:
sklearn.__version__

'1.2.2'

In [75]:
import imblearn
from imblearn.over_sampling import SMOTE

In [76]:
imblearn.__version__

'0.11.0'

In [77]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [78]:
X[0]

array([ 13. ,  13. ,  13. ,   0. ,  13. ,  13. ,  13. ,  13. , -76. ,
       -76. , -81. , -76. , -78.5, -76. , -76. ,  -7. ,  -7. , -12. ,
        -7. ,  -9.5,  -7. ,  -7. ,  12. ,  12. ,   7. ,  12. ,   9.5,
        12. ,  12. ])

In [79]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler()
fit_scaler = scaler.fit(X)
joblib.dump(fit_scaler, 'scaler.save')
X = fit_scaler.transform(X)
X.shape

(3524, 29)

In [80]:
X = torch.from_numpy(X).type(torch.float32)
y = torch.from_numpy(y).type(torch.float32)

In [81]:
X.shape

torch.Size([3524, 29])

In [82]:
X[0]

tensor([ 1.2869,  1.2773,  1.2949, -0.8119,  1.2809,  1.5356,  1.3002,  1.1853,
         1.3699,  1.3685,  1.0438,  1.3697,  1.2404,  1.3689,  1.3431,  0.5627,
         0.5623, -0.5295,  0.5599,  0.0665,  0.5606,  0.5251,  0.4544,  0.4446,
        -0.1923,  0.4467,  0.1804,  0.4452,  0.4011])

In [83]:
y.shape

torch.Size([3524])

In [84]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42
)

print("X_train:", X_train[:1],"\nX_train_shape:", X_train[:1].shape,"\nX_test:", X_test[:1],"\nX_test_shape:",X_test[:1].shape, "\ny_train:", y_train[:1],"\ny_test:", y_test[:1])

X_train: tensor([[ 0.3118,  1.2773, -0.3338,  1.4247,  0.3141,  0.1325,  0.3219,  0.6873,
          0.7814,  0.7799,  0.7831,  0.7811,  0.8137,  0.7802,  0.7539,  0.3437,
          0.3440, -0.0923,  0.3417,  0.1749,  0.3423,  0.3006,  1.0899,  1.0798,
          0.9498,  1.0833,  1.0753,  1.0811,  1.0443]]) 
X_train_shape: torch.Size([1, 29]) 
X_test: tensor([[ 1.9369,  1.5987,  1.6206, -0.2988,  1.6032,  1.8864,  1.6263,  1.6832,
          1.3046,  1.4338,  1.3046,  1.3043,  1.3389,  1.3035,  1.3431,  1.4389,
          1.4352,  1.4379,  1.4324,  1.4750,  1.4336,  1.4233,  1.7255,  1.7150,
          1.7112,  1.7198,  1.7785,  1.7170,  1.6875]]) 
X_test_shape: torch.Size([1, 29]) 
y_train: tensor([0.]) 
y_test: tensor([0.])


In [85]:
print("X_train:", X_train[:1])

X_train: tensor([[ 0.3118,  1.2773, -0.3338,  1.4247,  0.3141,  0.1325,  0.3219,  0.6873,
          0.7814,  0.7799,  0.7831,  0.7811,  0.8137,  0.7802,  0.7539,  0.3437,
          0.3440, -0.0923,  0.3417,  0.1749,  0.3423,  0.3006,  1.0899,  1.0798,
          0.9498,  1.0833,  1.0753,  1.0811,  1.0443]])


In [86]:
torch.save(X_train, "/tmp/X_train.pt")

In [87]:
X_train_loaded = torch.load("/tmp/X_train.pt")
print("X_train:", X_train_loaded[:1])

X_train: tensor([[ 0.3118,  1.2773, -0.3338,  1.4247,  0.3141,  0.1325,  0.3219,  0.6873,
          0.7814,  0.7799,  0.7831,  0.7811,  0.8137,  0.7802,  0.7539,  0.3437,
          0.3440, -0.0923,  0.3417,  0.1749,  0.3423,  0.3006,  1.0899,  1.0798,
          0.9498,  1.0833,  1.0753,  1.0811,  1.0443]])


In [88]:
type(X_train)

torch.Tensor

In [89]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [90]:
# Build model with non-linear activation function
from torch import nn
class InteruptionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=29, out_features=200)
        self.layer_2 = nn.Linear(in_features=200, out_features=100)
        self.layer_3 = nn.Linear(in_features=100, out_features=1)
        self.relu = nn.ReLU() # <- add in ReLU activation function
        # Can also put sigmoid in the model
        # This would mean you don't need to use it on the predictions
        # self.sigmoid = nn.Sigmoid()

    def forward(self, x):
      # Intersperse the ReLU activation function between layers
       return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))

model_3 = InteruptionModel().to(device)
print(model_3)

InteruptionModel(
  (layer_1): Linear(in_features=29, out_features=200, bias=True)
  (layer_2): Linear(in_features=200, out_features=100, bias=True)
  (layer_3): Linear(in_features=100, out_features=1, bias=True)
  (relu): ReLU()
)


In [91]:
#model_3.state_dict()

In [92]:
# Setup loss and optimizer
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_3.parameters(), lr=0.0001)

In [93]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [94]:
# Fit the model
torch.manual_seed(42)
epochs = 3500

# Put all data on target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)


for epoch in range(epochs):
    # 1. Forward pass
    y_logits = model_3(X_train).squeeze()

    y_pred = torch.round(torch.sigmoid(y_logits)) # logits -> prediction probabilities -> prediction labels

    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train) # BCEWithLogitsLoss calculates loss using logits
    acc = accuracy_fn(y_true=y_train,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_3.eval()
    with torch.no_grad():
      # 1. Forward pass
        test_logits = model_3(X_test).squeeze()
        #print(test_logits.shape)
        test_pred = torch.round(torch.sigmoid(test_logits)) # logits -> prediction probabilities -> prediction labels
        # 2. Calcuate loss and accuracy
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_true=y_test,
                             y_pred=test_pred)


    # Print out what's happening
    if epoch % 500 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.69271, Accuracy: 50.69% | Test Loss: 0.68669, Test Accuracy: 55.04%
Epoch: 500 | Loss: 0.47577, Accuracy: 76.98% | Test Loss: 0.49363, Test Accuracy: 75.46%
Epoch: 1000 | Loss: 0.36996, Accuracy: 83.54% | Test Loss: 0.43959, Test Accuracy: 80.28%
Epoch: 1500 | Loss: 0.26551, Accuracy: 89.68% | Test Loss: 0.41142, Test Accuracy: 83.26%
Epoch: 2000 | Loss: 0.17830, Accuracy: 94.40% | Test Loss: 0.40736, Test Accuracy: 85.39%
Epoch: 2500 | Loss: 0.12169, Accuracy: 96.59% | Test Loss: 0.43225, Test Accuracy: 86.24%
Epoch: 3000 | Loss: 0.08707, Accuracy: 97.69% | Test Loss: 0.46989, Test Accuracy: 85.96%


In [95]:
model_3.eval()
with torch.no_grad():
     y_preds = torch.round(torch.sigmoid(model_3(X_test))).squeeze()

In [96]:
y_preds.shape,y_test.shape

(torch.Size([705]), torch.Size([705]))

In [97]:
if device == "cuda":
  predictions = y_preds.cpu().numpy() #if it is cuda, then this, otherwise y_pred.numpy()
  true_labels = y_test.cpu().numpy()
else:
  predictions = y_preds.numpy()
  true_labels = y_test.numpy()

In [98]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score,fbeta_score

print("=== Confusion Matrix ===")
print(confusion_matrix(true_labels, predictions))
print('\n')


print("=== Score ===")
accuracy = accuracy_score(true_labels, predictions)
print('Accuracy: %f' % accuracy)

precision = precision_score(true_labels,  predictions, average='weighted')
print('Precision: %f' % precision)
recall = recall_score(true_labels, predictions, average='weighted')
print('Recall: %f' % recall)

microf1 = f1_score(true_labels, predictions, average='micro')
print('Micro F1 score: %f' % microf1)
macrof1 = f1_score(true_labels, predictions, average='macro')
print('Macro F1 score: %f' % macrof1)

=== Confusion Matrix ===
[[318  52]
 [ 43 292]]


=== Score ===
Accuracy: 0.865248
Precision: 0.865657
Recall: 0.865248
Micro F1 score: 0.865248
Macro F1 score: 0.865065


In [99]:
target_names = ['No-Stall', 'Stall']
# Print precision-recall report
print(classification_report(true_labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

    No-Stall       0.88      0.86      0.87       370
       Stall       0.85      0.87      0.86       335

    accuracy                           0.87       705
   macro avg       0.86      0.87      0.87       705
weighted avg       0.87      0.87      0.87       705



In [100]:
#Done

In [104]:
import torch
import os
from torch import nn
import joblib
from sklearn.preprocessing import StandardScaler

def preprocess(data):
    """
    Transform raw input into model input data.
    """
    try:
        # Log the incoming data for debugging
        logger.info(f"Received data: {data}")

        # Load scaler
        scaler = StandardScaler()
        scaler = joblib.load('scaler.save')

        tensor_list = []
        for item in data:
            item = scaler.transform([item['data']])
            tensor_data = torch.tensor(item, dtype=torch.float32)  # Each instance as a tensor
            tensor_list.append(tensor_data)
        # Stack all tensors along a new dimension to create a single tensor
        combined_tensor = torch.cat(tensor_list, dim=0)
        logger.info("Input data preprocessed successfully")
        return combined_tensor
    except Exception as e:
        logger.error(f"Error during preprocessing: {str(e)}")
        raise ValueError("Failed to preprocess input data")

def inference(model_input):
    """
    Perform model inference.
    """
    try:
        inference_list = []
        for tensor_data in model_input:
            with torch.no_grad():
                output = torch.round(torch.sigmoid(model_3(tensor_data))).squeeze()
            inference = output.cpu().numpy().tolist()
            inference_list.append(output)
        logger.info("Inference performed successfully")
        return inference_list
    except Exception as e:
        logger.error(f"Error during inference: {str(e)}")
        raise RuntimeError("Inference failed")

def postprocess(inference_output):
    """
    Convert model output to a list of predictions.
    """
    try:
        # Process each item in the batch
        result_list = []
        for result in inference_output:
            if result > 0:
                result_list.append("Stall")
            else:
                result_list.append("No Stall")
        logger.info("Output postprocessed successfully")
        return result_list
    except Exception as e:
        logger.error(f"Error during postprocessing: {str(e)}")
        raise ValueError("Failed to postprocess output data")

def handle(data):
    """
    Handle a prediction request.
    """
    try:
        model_input = preprocess(data)
        model_output = inference(model_input)
        return postprocess(model_output)
    except Exception as e:
        logger.error(f"Error during handle: {str(e)}")
        return [str(e)]

In [106]:
no_data = [{'data': [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}]
stall_data = [{'data': [14,14,14,0,14,14,14,14,-99,-99,-99,-99,-99,-99,-99,-5,-10,-10,-10,-10,-10,-7.5,17,17,17,17,17,17,17]}]
m_data = [{'data': [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}, {'data': [14,14,14,0,14,14,14,14,-99,-99,-99,-99,-99,-99,-99,-5,-10,-10,-10,-10,-10,-7.5,17,17,17,17,17,17,17]}]

handle(m_data)

INFO:__main__:Received data: [{'data': [13, 13, 13, 0, 13, 13, 13, 13, -76, -76, -81, -76, -78.5, -76, -76, -7, -7, -12, -7, -9.5, -7, -7, 12, 12, 7, 12, 9.5, 12, 12]}, {'data': [14, 14, 14, 0, 14, 14, 14, 14, -99, -99, -99, -99, -99, -99, -99, -5, -10, -10, -10, -10, -10, -7.5, 17, 17, 17, 17, 17, 17, 17]}]
INFO:__main__:Input data preprocessed successfully
INFO:__main__:Inference performed successfully
INFO:__main__:Output postprocessed successfully


['No Stall', 'Stall']